In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import regex as re
import string

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding

import warnings 
warnings.filterwarnings(action = 'ignore')

In [7]:
reddit = pd.read_csv('reddit_comments.csv', index_col=0)

reddit.shape

(1010714, 2)

In [8]:
# custom tokenizer function

def tokenizer(document):
    # get values from series
    
    #removing punctuation
    for punc in string.punctuation:
        document = document.replace(punc, '')
        # removing numbers and setting all documents to lowercase    
    document = re.sub("\d+", "", document).lower()
        # splitting documents and appending tokens list
    tokens = document.split(' ')
        
    
        

    return tokens

In [9]:
def vectorizer(document):

    tokens = tokenizer(document)
    
    
    size = model.vector_size
    word_vec_doc = np.zeros(size)
    count = 1
    

    for word in tokens:
        if word in model:
            count +=1
            word_vec_doc += model[word]
    
    word_vec_doc = word_vec_doc / count
    
    
    return word_vec_doc

In [10]:
model = KeyedVectors.load('sarcasm.wordvectors')

In [11]:
reddit['vectors'] = reddit['comment'].apply(vectorizer)

In [12]:
X = reddit['vectors']
y = reddit['label']

In [44]:
# X =np.asarray(X).astype(np.float32)
# y =np.asarray(y).astype(np.float32)

In [14]:
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, test_size=0.25, random_state=42)

In [21]:
X_train = np.array(X_train.tolist())
y_train = np.array(y_train.tolist())
X_val = np.array(X_val.tolist())
y_val = np.array(y_val.tolist())

In [22]:
X_train.shape

(568526, 300)

In [23]:
vocab = len(model.wv.vocab)

size = model.vector_size

length = 300

weights = model.wv.vectors

In [24]:
lstm_nn = Sequential()

lstm_nn.add(Embedding(vocab, size, input_length = length, weights = [weights], trainable=False))

lstm_nn.add(LSTM(128))
lstm_nn.add(Dropout(0.25))

lstm_nn.add(Dense(64, activation='relu'))
lstm_nn.add(Dropout(0.25))

lstm_nn.add(Dense(32, activation='relu'))
lstm_nn.add(Dropout(0.25))

lstm_nn.add(Dense(1, activation='sigmoid'))

lstm_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 300)          852000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)               

In [25]:
lstm_nn.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy'])

In [28]:
history = lstm_nn.fit(X_train.astype(np.float32), y_train, epochs=50, validation_data=(X_val, y_val), batch_size=100)

Epoch 1/50


InvalidArgumentError:  indices[71,2] = -1 is not in [0, 2840)
	 [[node sequential_1/embedding_1/embedding_lookup (defined at \AppData\Local\Temp\ipykernel_640\996834544.py:1) ]] [Op:__inference_train_function_6849]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
 sequential_1/embedding_1/embedding_lookup/5593 (defined at \anaconda3\envs\deeplearning\lib\contextlib.py:113)

Function call stack:
train_function
